# Create hybrid DFA

## Parse Process Model to DFA

In [35]:
processDFAs = ()

## Create Multi-Process DFA

In [36]:
processDFAs = (
    ("process1", {"0","1","2"} ,{"A","B"}, {"0": {("A","1")}, "1":{("B","2")}}, {"0"}, {"2"}),
    ("process2", {"0","1","2","3"} ,{"C","D","E"} ,{"0": {("C","1")}, "1":{("D","2")}, "2":{("E","3")}}, {"0"}, {"3"}),
    )

dfaStates = set()
dfaInputs = set()
dfaTransitions = {}
dfaInits = set()
dfaAccepting = set()

for process in processDFAs:
    newDfaStates = set()
    newDfaInputs = set()
    newDfaTransitions = {}
    newDfaInits = set()
    newDfaAccepting = set()

    # Create initial states for the DFA
    for init in process[4]:
        if dfaInits:
            for i in dfaInits:
                newDfaInits.add((*i, init))
        else:
            newDfaInits.add((init))
    dfaInits = newDfaInits

    # Create accepting states for the DFA
    for accept in process[5]:
        if dfaAccepting:
            for a in dfaAccepting:
                newDfaAccepting.add((*a, accept))
        else:
            newDfaAccepting.add((accept))
    dfaAccepting = newDfaAccepting

    # Add inputs from new process to the DFA
    for input in process[2]:
        dfaInputs.add(input)


    # Create states and transitions for the DFA
    for state in process[1]:
        if dfaStates:
            for s in dfaStates:
                newDfaStates.add((*s, state))
                newDfaTransitions[(*s, state)] = set()
                #add new state to old transitions
                if s in dfaTransitions:
                    for t in dfaTransitions[s]:
                        newDfaTransitions[(*s, state)].add((t[0], (*t[1], state))) 
                #add new transitions from new state, with old states
                if state in process[3]:
                    for trans in process[3][state]:
                        newDfaTransitions[(*s, state)].add((trans[0],(*s,trans[1])))
        else:
            newDfaStates.add((state))
            if state in process[3]:
                newDfaTransitions[(state)] = set()
                for trans in process[3][state]:
                    newDfaTransitions[(state)].add((trans[0],trans[1]))

    dfaStates = newDfaStates
    dfaTransitions = newDfaTransitions

multi_process_dfa = ("multi_process_dfa",dfaStates, dfaInputs, dfaTransitions, dfaInits, dfaAccepting)

print("states: " + str((multi_process_dfa[1])))
print("inputs: " + str(multi_process_dfa[2]))
print("transitions: " + str(multi_process_dfa[3]))
print("initial states: " + str(multi_process_dfa[4]))
print("accepting states: " + str(multi_process_dfa[5]))

states: {('2', '0'), ('1', '3'), ('1', '0'), ('2', '1'), ('0', '0'), ('0', '1'), ('2', '2'), ('1', '1'), ('1', '2'), ('0', '3'), ('0', '2'), ('2', '3')}
inputs: {'C', 'A', 'B', 'D', 'E'}
transitions: {('2', '0'): {('C', ('2', '1'))}, ('0', '0'): {('C', ('0', '1')), ('A', ('1', '0'))}, ('1', '0'): {('B', ('2', '0')), ('C', ('1', '1'))}, ('2', '2'): {('E', ('2', '3'))}, ('0', '2'): {('A', ('1', '2')), ('E', ('0', '3'))}, ('1', '2'): {('B', ('2', '2')), ('E', ('1', '3'))}, ('2', '3'): set(), ('0', '3'): {('A', ('1', '3'))}, ('1', '3'): {('B', ('2', '3'))}, ('2', '1'): {('D', ('2', '2'))}, ('0', '1'): {('A', ('1', '1')), ('D', ('0', '2'))}, ('1', '1'): {('B', ('2', '1')), ('D', ('1', '2'))}}
initial states: {('0', '0')}
accepting states: {('2', '3')}


## Prepare Declare constraint templates

In [37]:
#existence(p), F(p)
existenceTemplate = [
    "",
    {"existence_1", "existence_2"},



    
    {}, #all possible inputs from the multi-process-dfa
    {
        "existence_1": {}, # p into existence_2
        "existence_2": {}, # empty
    },
    {"existence_1"},
    {"existence_2"}
]

def existenceDFA(id, p, processInputs):
    dfa = existenceTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["existence_1"] = set()
    dfa[3]["existence_2"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == p:
            dfa[3]["existence_1"].add((activity,"existence_2"))
    return dfa

#absence2(p), !F(p&XF(p))
absence2Template = [
    "",
    {"absence2_1", "absence2_2","absence2_3"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "absence2_1": {}, # p into absence2_2
        "absence2_2": {}, # p into absence2_3
        "absence2_3": {}, # empty
    },
    {"absence2_1"},
    {"absence2_1","absence2_2"}
]

def absence2DFA(id, p, processInputs):
    dfa = absence2Template
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["absence2_1"] = set()
    dfa[3]["absence2_2"] = set()
    dfa[3]["absence2_3"] = set() 
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == p:
            dfa[3]["absence2_1"].add((activity,"absence2_2"))
            dfa[3]["absence2_2"].add((activity,"absence2_3"))
    return dfa

#choice(p,q), F(p)|F(q)
choiceTemplate = [
    "",
    {"choice_1", "choice_2"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "choice_1": {}, # p or q into choice_2
        "choice_2": {}
    },
    {"choice_1"},
    {"choice_2"}
]

def choiceDFA(id, p, q, processInputs):
    dfa = choiceTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["choice_1"] = set()
    dfa[3]["choice_2"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == p or activity == q:
            dfa[3]["choice_1"].add((activity,"choice_2"))
    return dfa

#exc-choice(p,q), ((F(p) | F(q)) & !((F(p) & F(q))))
exc_choiceTemplate = [
    "",
    {"exc-choice_1", "exc-choice_2", "exc-choice_3", "exc-choice_4"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "exc-choice_1": {}, # q & !p into exc-choice_2, p & !q into exc-choice_3, p and q into  exc-choice_4
        "exc-choice_2": {}, # p into exc-choice_4
        "exc-choice_3": {}, # q into exc-choice_4
        "exc-choice_4": {}
    },
    {"exc-choice_1"},
    {"exc-choice_2","exc-choice_3"}
]

def exc_choiceDFA(id, p, q, processInputs):
    dfa = exc_choiceTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["exc-choice_1"] = set()
    dfa[3]["exc-choice_2"] = set()
    dfa[3]["exc-choice_3"] = set()
    dfa[3]["exc-choice_4"] = set()  
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == q and activity != p:
            dfa[3]["exc-choice_1"].add((activity,"exc-choice_2"))
        elif activity == p and activity != q:
            dfa[3]["exc-choice_1"].add((activity,"exc-choice_3"))
        elif activity == p and activity == q:
            dfa[3]["exc-choice_1"].add((activity,"exc-choice_4"))     
        if activity == p:
            dfa[3]["exc-choice_2"].add((activity,"exc-choice_4"))   
        if activity == q:
            dfa[3]["exc-choice_3"].add((activity,"exc-choice_4"))   
    return dfa

#resp-existence(p,q), (F(p) -> F(q))
resp_existenceTemplate = [
    "",
    {"resp-existence_1", "resp-existence_2", "resp-existence_3"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "resp-existence_1": {}, # q into resp-existence_2, p & !q into resp-existence_3
        "resp-existence_2": {}, # empty
        "resp-existence_3": {}, # q into resp-existence_2
    },
    {"resp-existence_1"},
    {"resp-existence_1","resp-existence_2"}
]

def resp_existenceDFA(id, p, q, processInputs):
    dfa = resp_existenceTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["resp-existence_1"] = set()
    dfa[3]["resp-existence_2"] = set()
    dfa[3]["resp-existence_3"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == q:
            dfa[3]["resp-existence_1"].add((activity,"resp-existence_2"))
        elif activity == p and activity != q:
            dfa[3]["resp-existence_1"].add((activity,"resp-existence_3"))
        if activity == q:
            dfa[3]["resp-existence_3"].add((activity,"resp-existence_2"))   
    return dfa

#coexistence(p,q), (F(p) <-> F(q))
coexistenceTemplate = [
    "",
    {"coexistence_1", "coexistence_2", "coexistence_3", "coexistence_4"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "coexistence_1": {}, # q & !p into coexistence_2, p & !q into coexistence_3, p and q into  coexistence_4
        "coexistence_2": {}, # p into coexistence_4
        "coexistence_3": {}, # q into coexistence_4
        "coexistence_4": {}
    },
    {"coexistence_1"},
    {"coexistence_1","coexistence_4"}
]

def coexistenceDFA(id, p, q, processInputs):
    dfa = coexistenceTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["coexistence_1"] = set()
    dfa[3]["coexistence_2"] = set()
    dfa[3]["coexistence_3"] = set()
    dfa[3]["coexistence_4"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == q and activity != p:
            dfa[3]["coexistence_1"].add((activity,"coexistence_2"))
        elif activity == p and activity != q:
            dfa[3]["coexistence_1"].add((activity,"coexistence_3"))
        elif activity == p and activity == q:
            dfa[3]["coexistence_1"].add((activity,"coexistence_4"))     
        if activity == p:
            dfa[3]["coexistence_2"].add((activity,"coexistence_4"))   
        if activity == q:
            dfa[3]["coexistence_3"].add((activity,"coexistence_4"))   
    return dfa

#response(p,q), G((p -> F(q)))
responseTemplate = [
    "",
    {"response_1", "response_2"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "response_1": {}, # p & !q into response_2
        "response_2": {}, # q into reponse_1
    },
    {"response_1"},
    {"response_1"}
]

def responseDFA(id, p, q, processInputs):
    dfa = responseTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["response_1"] = set()
    dfa[3]["response_2"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == p and activity != q:
            dfa[3]["response_1"].add((activity,"response_2"))
        if activity == q:
            dfa[3]["response_2"].add((activity,"response_1"))   
    return dfa

#precedence(p,q), (!(q) W p) = (!(q) U p) | G(!q)
precedenceTemplate = [
    "",
    {"precedence_1", "precedence_2", "precedence_3"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "precedence_1": {}, # p into precedence_2, q & !p into precedence_3
        "precedence_2": {}, # empty
        "precedence_3": {}, # empty
    },
    {"precedence_1"},
    {"precedence_1","precedence_2"}
]

def precedenceDFA(id, p, q, processInputs):
    dfa = precedenceTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["precedence_1"] = set()
    dfa[3]["precedence_2"] = set()
    dfa[3]["precedence_3"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == q and activity != p:
            dfa[3]["precedence_1"].add((activity,"precedence_3"))
        elif activity == p:
            dfa[3]["precedence_1"].add((activity,"precedence_2")) 
    return dfa

#succession(p,q), (G((p -> F(q))) & (!(q) W p)) = (G((p -> F(q))) & ((!(q) U p) | G(!(q))))
successionTemplate = [
    "",
    {"succession_1", "succession_2", "succession_3", "succession_4"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "succession_1": {}, # q & !p into succession_2, p & !q into succession_3, p and q into  succession_4
        "succession_2": {}, # empty
        "succession_3": {}, # q into succession_4
        "succession_4": {} # p & !q into succession_3 
    },
    {"succession_1"},
    {"succession_4"}
]

def successionDFA(id, p, q, processInputs):
    dfa = successionTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["succession_1"] = set()
    dfa[3]["succession_2"] = set()
    dfa[3]["succession_3"] = set()
    dfa[3]["succession_4"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == q and activity != p:
            dfa[3]["succession_1"].add((activity,"succession_2"))
        elif activity == p and activity != q:
            dfa[3]["succession_1"].add((activity,"succession_3"))
        elif activity == p and activity == q:
            dfa[3]["succession_1"].add((activity,"succession_4"))     
        if activity == q:
            dfa[3]["succession_3"].add((activity,"succession_4"))   
        if activity == p and activity != q:
            dfa[3]["succession_4"].add((activity,"succession_3"))   
    return dfa

#alt-response(p,q), G((p -> X((!(p) U q))))
alt_responseTemplate = [
    "",
    {"alt-response_1", "alt-response_2", "alt-response_3"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "alt-response_1": {}, # p into alt-response_2
        "alt-response_2": {}, # p & !q into alt-response_3, q & !p into into alt-response_1
        "alt-response_3": {}, # empty
    },
    {"alt-response_1"},
    {"alt-response_1"}
]

def alt_responseDFA(id, p, q, processInputs):
    dfa = alt_responseTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["alt-response_1"] = set()
    dfa[3]["alt-response_2"] = set()
    dfa[3]["alt-response_3"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == p:
            dfa[3]["alt-response_1"].add((activity,"alt-response_2"))
        if activity == p and activity != q:
            dfa[3]["alt-response_2"].add((activity,"alt-response_3"))   
        elif activity == q and activity != p:
            dfa[3]["alt-response_2"].add((activity,"alt-response_1"))   
    return dfa


#alt-precedence(p,q), ((!(q) W p) & G((q -> ~X((!(q) W p))))) = (((!(q) U p) | G(!(q))) & G((q -> WX(((!(q) U p) | G(!(q)))))))
alt_precedenceTemplate = [
    "",
    {"alt-precedence_1", "alt-precedence_2", "alt-precedence_3"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "alt-precedence_1": {}, # p & !q into alt-precedence_2, q & !p into alt-precedence_3
        "alt-precedence_2": {}, # q into alt-precedence_1
        "alt-precedence_3": {}, # empty
    },
    {"alt-precedence_1"},
    {"alt-precedence_1","alt-precedence_2"}
]

def alt_precedenceDFA(id, p, q, processInputs):
    dfa = alt_precedenceTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["alt-precedence_1"] = set()
    dfa[3]["alt-precedence_2"] = set()
    dfa[3]["alt-precedence_3"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == p and activity != q:
            dfa[3]["alt-precedence_1"].add((activity,"alt-precedence_2"))
        elif activity == q and activity != p:
            dfa[3]["alt-precedence_1"].add((activity,"alt-precedence_3"))
        if activity == q:
            dfa[3]["alt-precedence_2"].add((activity,"alt-precedence_1"))   
    return dfa

#alt-succession(p,q), G(p->X((!p)Uq))&((!q)Wp)&G(q->WX((!q)Wp)) = G(p->X((!p)Uq))&((!(q) U p) | G(!q)&G(q->WX((!(q) U p) | G(!q)))
#
#
# TO BE DONE
#

#chain-response(p,q), G((p -> X(q)))
chain_responseTemplate = [
    "",
    {"chain-response_1", "chain-response_2", "chain-response_3"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "chain-response_1": {}, # p into chain-response_2
        "chain-response_2": {}, # !q into chain-response_3, q & !p into chain-response_1
        "chain-response_3": {}, # empty
    },
    {"chain-response_1"},
    {"chain-response_1"}
]

def chain_responseDFA(id, p, q, processInputs):
    dfa = chain_responseTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["chain-response_1"] = set()
    dfa[3]["chain-response_2"] = set()
    dfa[3]["chain-response_3"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == p:
            dfa[3]["chain-response_1"].add((activity,"chain-response_2"))
        if activity != q:
            dfa[3]["chain-response_2"].add((activity,"chain-response_3"))
        elif activity == q and activity != p:
            dfa[3]["chain-response_2"].add((activity,"chain-response_1"))
    return dfa

#chain-precedence(p,q), G((X(q) -> p))
chain_precedenceTemplate = [
    "",
    {"chain-precedence_1", "chain-precedence_2", "chain-precedence_3"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "chain-precedence_1": {}, # !p into chain-precedence_2
        "chain-precedence_2": {}, # q into chain-precedence_3, p & !q into chain-precedence_1
        "chain-precedence_3": {}, # empty
    },
    {"chain-precedence_1"},
    {"chain-precedence_1","chain-precedence_2"}
]

def chain_precedenceDFA(id, p, q, processInputs):
    dfa = chain_precedenceTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["chain-precedence_1"] = set()
    dfa[3]["chain-precedence_2"] = set()
    dfa[3]["chain-precedence_3"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity != p:
            dfa[3]["chain-precedence_1"].add((activity,"chain-precedence_2"))
        if activity == q:
            dfa[3]["chain-precedence_2"].add((activity,"chain-precedence_3"))
        elif activity == p and activity != q:
            dfa[3]["chain-precedence_2"].add((activity,"chain-precedence_1"))
    return dfa


#chain-succession(p,q), G((p <-> X(q)))
chain_successionTemplate = [
    "",
    {"chain-succession_1", "chain-succession_2", "chain-succession_3", "chain-succession_4"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "chain-succession_1": {}, # !p into chain-succession_2, p into chain-succession_3
        "chain-succession_2": {}, # p & !q into chain-succession_3, q into chain-succession_4
        "chain-succession_3": {}, # q & !p into chain-succession_2, !q into chain-succession_4
        "chain-succession_4": {} # empty
    },
    {"chain-succession_1"},
    {"chain-succession_1","chain-succession_2"}
]

def chain_successionDFA(id, p, q, processInputs):
    dfa = chain_successionTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["chain-succession_1"] = set()
    dfa[3]["chain-succession_2"] = set()
    dfa[3]["chain-succession_3"] = set()
    dfa[3]["chain-succession_4"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity != p:
            dfa[3]["chain-succession_1"].add((activity,"chain-succession_2"))
        elif activity == p:
            dfa[3]["chain-succession_1"].add((activity,"chain-succession_3"))
        if activity == p and activity != q:
            dfa[3]["chain-succession_2"].add((activity,"chain-succession_3"))
        elif activity == q:
            dfa[3]["chain-succession_2"].add((activity,"chain-succession_4"))
        if activity == q and activity != p:
            dfa[3]["chain-succession_3"].add((activity,"chain-succession_2"))
        elif activity != q:
            dfa[3]["chain-succession_3"].add((activity,"chain-succession_4"))
    return dfa

#not-coexistence(p,q), !((F(p) & F(q)))
not_coexistenceTemplate = [
    "",
    {"not-coexistence_1", "not-coexistence_2", "not-coexistence_3", "not-coexistence_4"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "not-coexistence_1": {}, # q & !p into not-coexistence_2, p & !q into not-coexistence_3, p & q into not-coexistence_4
        "not-coexistence_2": {}, # p into not-coexistence_4
        "not-coexistence_3": {}, # q into not-coexistence_4
        "not-coexistence_4": {} # empty
    },
    {"not-coexistence_1"},
    {"not-coexistence_1","not-coexistence_2","not-coexistence_3"}
]

def not_coexistenceDFA(id, p, q, processInputs):
    dfa = not_coexistenceTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["not-coexistence_1"] = set()
    dfa[3]["not-coexistence_2"] = set()
    dfa[3]["not-coexistence_3"] = set()
    dfa[3]["not-coexistence_4"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == q and activity != p:
            dfa[3]["not-coexistence_1"].add((activity,"not-coexistence_2"))
        elif activity == p and activity != q:
            dfa[3]["not-coexistence_1"].add((activity,"not-coexistence_3"))
        elif activity == p and activity == q:
            dfa[3]["not-coexistence_1"].add((activity,"not-coexistence_4"))
        if activity == p:
            dfa[3]["not-coexistence_2"].add((activity,"not-coexistence_4"))
        if activity == q:
            dfa[3]["not-coexistence_3"].add((activity,"not-coexistence_4"))
    return dfa

#neg-succession(p,q), G((p -> !(F(q))))
neg_successionTemplate = [
    "",
    {"neg-succession_1", "neg-succession_2", "neg-succession_3"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "neg-succession_1": {}, # p & !q into neg-succession_2, p & q into neg-succession_3
        "neg-succession_2": {}, # q into neg-succession_3
        "neg-succession_3": {}, # empty
    },
    {"neg-succession_1"},
    {"neg-succession_1","neg-succession_2"}
]

def neg_successionDFA(id, p, q, processInputs):
    dfa = neg_successionTemplate
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["neg-succession_1"] = set()
    dfa[3]["neg-succession_2"] = set()
    dfa[3]["neg-succession_3"] = set()
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == p and activity != q:
            dfa[3]["neg-succession_1"].add((activity,"neg-succession_2"))
        elif activity == p and activity == q:
            dfa[3]["neg-succession_1"].add((activity,"neg-succession_3"))
        if activity == q:
            dfa[3]["neg-succession_2"].add((activity,"neg-succession_3"))
    return dfa

#neg-chain-succession(p,q), (G((p -> WX(!(q)))) & G((q -> WX(!(p)))))
neg_chain_successionTemplate = [
    "",
    {"neg-chain-succession_1", "neg-chain-succession_2", "neg-chain-succession_3", "neg-chain-succession_4", "neg-chain-succession_5"},
    {}, #all possible inputs from the multi-process-dfa
    {
        "neg-chain-succession_1": {}, # q & !p into neg-chain-succession_2, p & !q into neg-chain-succession_3, p & q into neg-chain-succession_4
        "neg-chain-succession_2": {}, # p into neg-chain-succession_5, !p & !q into neg-chain-succession_1
        "neg-chain-succession_3": {}, # q into neg-chain-succession_5, !p & !q into neg-chain-succession_1
        "neg-chain-succession_4": {}, # p | q into neg-chain-succession_5, !p & !q into neg-chain-succession_1
        "neg-chain-succession_5": {} # empty
    },
    {"neg-chain-succession_1"},
    {"neg-chain-succession_1","neg-chain-succession_2","neg-chain-succession_3"}
]

def neg_chain_successionDFA(id, p, q, processInputs):
    dfa = neg_chain_successionTemplate.copy()
    dfa[0] = id
    dfa[2] = set()
    dfa[3]["neg-chain-succession_1"] = set()
    dfa[3]["neg-chain-succession_2"] = set()
    dfa[3]["neg-chain-succession_3"] = set()
    dfa[3]["neg-chain-succession_4"] = set()
    dfa[3]["neg-chain-succession_5"] = set()    
    for activity in processInputs:
        dfa[2].add(activity)
        if activity == q and activity != p:
            dfa[3]["neg-chain-succession_1"].add((activity,"neg-chain-succession_2"))
        elif activity == p and activity != q:
            dfa[3]["neg-chain-succession_1"].add((activity,"neg-chain-succession_3"))
        elif activity == p and activity == q:
            dfa[3]["neg-chain-succession_1"].add((activity,"neg-chain-succession_4"))
        if activity == p:
            dfa[3]["neg-chain-succession_2"].add((activity,"neg-chain-succession_5"))
        elif activity != p and activity != q:
            dfa[3]["neg-chain-succession_2"].add((activity,"neg-chain-succession_1"))
        if activity == q:
            dfa[3]["neg-chain-succession_3"].add((activity,"neg-chain-succession_5"))
        elif activity != p and activity != q:
            dfa[3]["neg-chain-succession_3"].add((activity,"neg-chain-succession_1")) 
        if activity == p or activity == q:
            dfa[3]["neg-chain-succession_4"].add((activity,"neg-chain-succession_5"))
        elif activity != p and activity != q:
            dfa[3]["neg-chain-succession_4"].add((activity,"neg-chain-succession_1")) 
    return dfa


## Create Constraint DFA

In [38]:
constraintDFAs = []

constraintsFromModel = {
    ("existence","testID1", "A"),
    ("absence2","testID2", "A"),
    ("choice","testID3", "A","C"),
    ("exc-choice","testID4", "A","C"),
    ("resp-existence","testID5", "A","C"),
    ("coexistence","testID6", "A","C"),
    ("response","testID7", "A","C"),
    ("precedence","testID8", "A","C"),
    ("succession","testID9", "A","C"),
    ("alt-response","testID10", "A","C"),
    ("alt-precedence","testID11", "A","C"),
    # ("alt-succession","testID12", "A","C"),
    ("chain-response","testID13", "A","C"),
    ("chain-precedence","testID14", "A","C"),
    ("chain-succession","testID15", "A","C"),
    ("not-coexistence","testID16", "A","C"),
    ("neg-succession","testID17", "A","C"),
    ("neg-chain-succession","testID18", "A","C")
    } # (type, p) / (type, p, q)

for constraint in constraintsFromModel:
    newDFA = ()
    if constraint[0] == "existence":
        newDFA = existenceDFA(constraint[1],constraint[2],multi_process_dfa[2])
    elif constraint[0] == "absence2":
        newDFA = absence2DFA(constraint[1],constraint[2],multi_process_dfa[2])
    elif constraint[0] == "choice":
        newDFA = choiceDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "exc-choice":
        newDFA = exc_choiceDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "resp-existence":
        newDFA = resp_existenceDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "coexistence":
        newDFA = coexistenceDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "response":
        newDFA = responseDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "precedence":
        newDFA = precedenceDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "succession":
        newDFA = successionDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "alt-response":
        newDFA = alt_responseDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "alt-precedence":
        newDFA = alt_precedenceDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    # elif constraint[0] == "alt-succession":
    #     newDFA = alt_successionDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "chain-response":
        newDFA = chain_responseDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "chain-precedence":
        newDFA = chain_precedenceDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "chain-succession":
        newDFA = chain_successionDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "not-coexistence":
        newDFA = not_coexistenceDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "neg-succession":
        newDFA = neg_successionDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])
    elif constraint[0] == "neg-chain-succession":
        newDFA = neg_chain_successionDFA(constraint[1],constraint[2],constraint[3],multi_process_dfa[2])

    constraintDFAs.append(newDFA)
    

# array instead of set to be able to access by index
# important for checking the accepting states during colouring
# constraintDFAs = [
#     ("constraints1",
#      {"state_1","state_2","state_3"},
#      {"A","B","C","D","E"},
#      {"state_1": {("A","state_2")}, 
#       "state_2":{("C","state_1"),("A","state_3"),("B","state_3"),("D","state_3"),("E","state_3")},"state_3":{}},
#      {"state_1"},
#      {"state_1"}),
#     ("constraints2",
#      {"state_4","state_5","state_6"},
#      {"A","B","C","D","E"},
#      {"state_4": {("A","state_5")}, 
#       "state_5":{("C","state_4"),("A","state_6"),("B","state_6"),("D","state_6"),("E","state_6")},"state_6":{}},
#      {"state_4"},
#      {"state_6"}),
#     ("constraints3",
#      {"state_7","state_8"},
#      {"A","B","C","D","E"},
#      {"state_7": {("A","state_8"),("B","state_8")}, 
#       "state_8":{("C","state_7"),("B","state_8"),("D","state_7"),("E","state_7")}},
#      {"state_7"},
#      {"state_8"}),
# ]

for constraint in constraintDFAs:
    print("Name: ", constraint[0])
    print("States: ", constraint[1])
    print("Inputs: ", constraint[2])
    print("Transitions: ", constraint[3])
    print("Initial States: ", constraint[4])
    print("Accepting States: ", constraint[5])
    print("\n")

Name:  testID14
States:  {'chain-precedence_3', 'chain-precedence_1', 'chain-precedence_2'}
Inputs:  {'C', 'A', 'B', 'D', 'E'}
Transitions:  {'chain-precedence_1': {('C', 'chain-precedence_2'), ('B', 'chain-precedence_2'), ('E', 'chain-precedence_2'), ('D', 'chain-precedence_2')}, 'chain-precedence_2': {('A', 'chain-precedence_1'), ('C', 'chain-precedence_3')}, 'chain-precedence_3': set()}
Initial States:  {'chain-precedence_1'}
Accepting States:  {'chain-precedence_1', 'chain-precedence_2'}


Name:  testID8
States:  {'precedence_3', 'precedence_1', 'precedence_2'}
Inputs:  {'C', 'A', 'B', 'D', 'E'}
Transitions:  {'precedence_1': {('A', 'precedence_2'), ('C', 'precedence_3')}, 'precedence_2': set(), 'precedence_3': set()}
Initial States:  {'precedence_1'}
Accepting States:  {'precedence_1', 'precedence_2'}


Name:  testID10
States:  {'alt-response_1', 'alt-response_3', 'alt-response_2'}
Inputs:  {'C', 'A', 'B', 'D', 'E'}
Transitions:  {'alt-response_1': {('A', 'alt-response_2')}, 'alt-

## Create Global Hybrid DFA

In [39]:
hybrid_DFA = multi_process_dfa
hybridStates = set()
hybridInputs = set()
hybridTransitions = {}
hybridInits = set()
hybridAccepting = set()

for constraint in constraintDFAs:
    newHybridStates = set()
    newHybridTransitions = {}
    newHybridInits = set()
    newHybridAccepting = set()
    usedStates = set()

    # Create initial states for the DFA
    for init in constraint[4]:
        for m in hybrid_DFA[4]:
            newHybridInits.add((*m, init))
            newHybridStates.add((*m, init))
    hybridInits = newHybridInits
    #print("Inits: ", hybridInits)

    #Create accepting states for the DFA
    for accept in hybrid_DFA[5]:
        for a in constraint[5]:
            newHybridAccepting.add((*accept, a))
    hybridAccepting = newHybridAccepting
    #print("Accpeting: ", hybridAccepting)

    #Add inputs from new process to the DFA
    for input in hybrid_DFA[2]:
        hybridInputs.add(input)
    for input in constraint[2]:
        hybridInputs.add(input)
    #print("Inputs: ", hybridInputs)

    # Create states and transitions for the DFA
    while usedStates != newHybridStates:
        #print("")
        #print("new iteration")
        for state in newHybridStates.copy():
            last = len(state) - 1
            if state not in usedStates:
                #print("hybrid", hybrid_DFA)
                allTransDFA = set()
                for trans in hybrid_DFA[3][state[:-1]]:
                    allTransDFA.add(trans[0])
                #print("")
                #print(state, "allTransDFA: ", allTransDFA) 

                allTransConstraint = set()
                for trans in constraint[3][state[last]]:
                    allTransConstraint.add(trans[0])
                #print(state, "allTransConstraint: ", allTransConstraint)

                for trans in hybrid_DFA[3][state[:-1]]:
                    for t in constraint[3][state[last]]:
                        if trans[0] == t[0] and t[0] in allTransDFA:
                            newHybridStates.add((*trans[1],t[1]))
                            if state not in newHybridTransitions:
                                newHybridTransitions[state] = set()
                            newHybridTransitions[state].add((trans[0],(*trans[1],t[1])))
                            #print("IF", "new state: ", (*trans[1],t[1]), "new transition: ",state,(trans[0],(*trans[1],t[1])))

                        elif trans[0] not in allTransConstraint:
                            newHybridStates.add((*trans[1],state[last]))
                            if state not in newHybridTransitions:
                                newHybridTransitions[state] = set()
                            newHybridTransitions[state].add((trans[0],(*trans[1],state[last])))
                            #print("ELSE", "new state: ", (*trans[1],state[last]), "new transition: ",state,(trans[0],(*trans[1],state[last])))
                    if not allTransConstraint:
                        newHybridStates.add((*trans[1],state[last]))
                        if state not in newHybridTransitions:
                            newHybridTransitions[state] = set()
                        newHybridTransitions[state].add((trans[0],(*trans[1],state[last])))
                        #print("ELSE", "new state: ", (*trans[1],state[last]), "new transition: ",state,(trans[0],(*trans[1],state[last])))
                        
                if state not in newHybridTransitions:
                    newHybridTransitions[state] = set()
                usedStates.add(state)   

    hybridStates = newHybridStates
    hybridTransitions = newHybridTransitions

    hybrid_DFA = ("hybridDFA",hybridStates, hybridInputs, hybridTransitions, hybridInits, hybridAccepting)
    # print(len(hybrid_DFA[0]),len(hybrid_DFA[2]))
    # print("states: " + str((hybrid_DFA[0])))
    # print("inputs: " + str(hybrid_DFA[1]))
    # print("transitions: ")
    # for i in hybrid_DFA[2]:
    #     print(i, hybrid_DFA[2][i])
    # print("initial states: " + str(hybrid_DFA[3]))
    # print("accepting states: " + str(hybrid_DFA[4]))

#remove states from accepting list that do not exist in DFA
acceptingExists = False
for accept in hybrid_DFA[5]:
    hybridCopy = hybrid_DFA[5].copy()
    if accept in hybridCopy:
        acceptingExists = True
    else:
        hybrid_DFA[5].remove(accept)

print(len(hybrid_DFA[1]),len(hybrid_DFA[3]))
print("states: " + str((hybrid_DFA[1])))
print("inputs: " + str(hybrid_DFA[2]))
print("transitions: ")
for i in hybrid_DFA[3]:
    print(i, hybrid_DFA[3][i])
print("initial states: " + str(hybrid_DFA[4]))
print("accepting states: " + str(hybrid_DFA[5]))

26 26
states: {('0', '3', 'chain-precedence_2', 'precedence_3', 'alt-response_1', 'neg-chain-succession_1', 'existence_1', 'succession_2', 'choice_2', 'response_1', 'chain-response_1', 'absence2_1', 'not-coexistence_2', 'neg-succession_1', 'resp-existence_2', 'alt-precedence_3', 'exc-choice_2', 'chain-succession_2', 'coexistence_2'), ('1', '3', 'chain-precedence_2', 'precedence_2', 'alt-response_1', 'neg-chain-succession_5', 'existence_2', 'succession_4', 'choice_2', 'response_1', 'chain-response_1', 'absence2_2', 'not-coexistence_4', 'neg-succession_3', 'resp-existence_2', 'alt-precedence_1', 'exc-choice_4', 'chain-succession_2', 'coexistence_4'), ('2', '2', 'chain-precedence_3', 'precedence_2', 'alt-response_1', 'neg-chain-succession_1', 'existence_2', 'succession_4', 'choice_2', 'response_1', 'chain-response_3', 'absence2_2', 'not-coexistence_4', 'neg-succession_3', 'resp-existence_2', 'alt-precedence_1', 'exc-choice_4', 'chain-succession_4', 'coexistence_4'), ('1', '1', 'chain-prec

# Create Monitor

In [40]:
class ColoredDFA:
    def __init__(self, current, states, alphabet, transition_function, init_state, accept_states, colors):
        self.current = current
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.init_state = init_state
        self.accept_states = accept_states
        self.colors = colors

## Prepare DFA Data

In [41]:
numberProcesses = len(processDFAs)

colouredStates = {}

stateColours = [None] * len(constraintDFAs)

# Colours for DFA states
satisfied = "satisfied" #0
temporary_satisfied = "temporary_satisfied" #1
temporary_violated = "temporary_violated" #2
violated = "violated" #3


## Colour states on constraint satisfaction

In [42]:
for state in hybrid_DFA[1]:
    colouredStates[state] = stateColours.copy()
    for index, constraint in enumerate(constraintDFAs):
        if state[numberProcesses+index] in constraint[5]:
            colouredStates[state][index] = {constraint[0]: satisfied}
        else:
            colouredStates[state][index] = {constraint[0]: violated}

print(colouredStates)

{('0', '3', 'chain-precedence_2', 'precedence_3', 'alt-response_1', 'neg-chain-succession_1', 'existence_1', 'succession_2', 'choice_2', 'response_1', 'chain-response_1', 'absence2_1', 'not-coexistence_2', 'neg-succession_1', 'resp-existence_2', 'alt-precedence_3', 'exc-choice_2', 'chain-succession_2', 'coexistence_2'): [{'testID14': 'satisfied'}, {'testID8': 'violated'}, {'testID10': 'satisfied'}, {'testID18': 'satisfied'}, {'testID1': 'violated'}, {'testID9': 'violated'}, {'testID3': 'satisfied'}, {'testID7': 'satisfied'}, {'testID13': 'satisfied'}, {'testID2': 'satisfied'}, {'testID16': 'satisfied'}, {'testID17': 'satisfied'}, {'testID5': 'satisfied'}, {'testID11': 'violated'}, {'testID4': 'satisfied'}, {'testID15': 'satisfied'}, {'testID6': 'violated'}], ('1', '3', 'chain-precedence_2', 'precedence_2', 'alt-response_1', 'neg-chain-succession_5', 'existence_2', 'succession_4', 'choice_2', 'response_1', 'chain-response_1', 'absence2_2', 'not-coexistence_4', 'neg-succession_3', 'resp-

## Colour states on reachability

In [43]:
def changeColours(index, constraintName, currentState, currentColour):
    #print("test")
    if hybrid_DFA[3][currentState]:
        #print(currentState, "has trans")
        for target in hybrid_DFA[3][currentState]:
            #print(target[1])
            changeColours(index, constraintName, target[1], colouredStates[target[1]][index][constraintName])
        reachableColours = set()
        for target in hybrid_DFA[3][currentState]:
            reachableColours.add(colouredStates[target[1]][index][constraintName])
        
        if currentColour == satisfied:
            if violated in reachableColours:
                colouredStates[currentState][index][constraintName] = temporary_satisfied
            elif temporary_satisfied in reachableColours:
                colouredStates[currentState][index][constraintName] = temporary_satisfied
            else:
                colouredStates[currentState][index][constraintName] = satisfied
        elif currentColour == violated:
            if satisfied in reachableColours:
                colouredStates[currentState][index][constraintName] = temporary_violated
            elif temporary_violated in reachableColours:
                colouredStates[currentState][index][constraintName] = temporary_violated
            else:
                colouredStates[currentState][index][constraintName] = violated
        # elif currentColour == temporary_satisfied:
        #     print("ERROR", currentState, temporary_satisfied)
        # elif currentColour == temporary_violated:
        #     print("ERROR", currentState, temporary_violated)
    else:
        #print(currentState, "has no trans")
        colouredStates[currentState][index][constraint[0]] = currentColour


print(len(colouredStates))

for index, constraint in enumerate(constraintDFAs):
    for init in hybrid_DFA[4]:
        #print(colouredStates[init][index][constraint[0]])
        changeColours(index, constraint[0], init, colouredStates[init][index][constraint[0]])


print(len(colouredStates))
for r in colouredStates:
    print(r, colouredStates[r])

26
26
('0', '3', 'chain-precedence_2', 'precedence_3', 'alt-response_1', 'neg-chain-succession_1', 'existence_1', 'succession_2', 'choice_2', 'response_1', 'chain-response_1', 'absence2_1', 'not-coexistence_2', 'neg-succession_1', 'resp-existence_2', 'alt-precedence_3', 'exc-choice_2', 'chain-succession_2', 'coexistence_2') [{'testID14': 'satisfied'}, {'testID8': 'violated'}, {'testID10': 'temporary_satisfied'}, {'testID18': 'satisfied'}, {'testID1': 'temporary_violated'}, {'testID9': 'violated'}, {'testID3': 'satisfied'}, {'testID7': 'temporary_satisfied'}, {'testID13': 'temporary_satisfied'}, {'testID2': 'satisfied'}, {'testID16': 'temporary_satisfied'}, {'testID17': 'satisfied'}, {'testID5': 'satisfied'}, {'testID11': 'violated'}, {'testID4': 'temporary_satisfied'}, {'testID15': 'temporary_satisfied'}, {'testID6': 'temporary_violated'}]
('1', '3', 'chain-precedence_2', 'precedence_2', 'alt-response_1', 'neg-chain-succession_5', 'existence_2', 'succession_4', 'choice_2', 'response_1'

In [47]:
# create coloredDFA from hybridDFA
current = ""
for i in hybrid_DFA[4]:
    current = i
states = hybrid_DFA[1]
alphabet = hybrid_DFA[2]
transitions = hybrid_DFA[3]
init = ""
for i in hybrid_DFA[4]:
    init = i
accepting = hybrid_DFA[5]
colors = colouredStates

colored_dfa = ColoredDFA(current,states,alphabet,transitions,init,accepting,colors)
print(colored_dfa.transition_function[colored_dfa.current])

{('C', ('0', '1', 'chain-precedence_2', 'precedence_3', 'alt-response_1', 'neg-chain-succession_2', 'existence_1', 'succession_2', 'choice_2', 'response_1', 'chain-response_1', 'absence2_1', 'not-coexistence_2', 'neg-succession_1', 'resp-existence_2', 'alt-precedence_3', 'exc-choice_2', 'chain-succession_2', 'coexistence_2')), ('A', ('1', '0', 'chain-precedence_1', 'precedence_2', 'alt-response_2', 'neg-chain-succession_3', 'existence_2', 'succession_3', 'choice_2', 'response_2', 'chain-response_2', 'absence2_2', 'not-coexistence_3', 'neg-succession_2', 'resp-existence_3', 'alt-precedence_2', 'exc-choice_3', 'chain-succession_3', 'coexistence_3'))}


# Create JSON for Monitor